In [1]:
import PySaRLAC as sl
import random
import math
import numpy as np
import h5py
random.seed(1234)

In [2]:
#For reference, this is the mapping between operator index and operator name
idx_op_map = ["PiPiGnd","PiPiExc","Sigma"]
op_idx_map = dict()
for i in range(len(op_idx_map)):
    op_idx_map[idx_op_map[i]] = i


In [3]:
f = h5py.File('combined_data.hdf5','r')

In [4]:
f.keys() #should see jackknife (binned) and block-double jackknife data

<KeysViewHDF5 ['bdj_data', 'j_data']>

In [5]:
#These are the operator pairs in the data set
jdata = f['j_data']
con = jdata['contains']['entries']
for e in con.keys():
    op1 = con[e].attrs["first"][0]
    op2 = con[e].attrs["second"][0]
    print(op1,op2,idx_op_map[op1],idx_op_map[op2])

0 0 PiPiGnd PiPiGnd
0 1 PiPiGnd PiPiExc
0 2 PiPiGnd Sigma
1 1 PiPiExc PiPiExc
1 2 PiPiExc Sigma
2 2 Sigma Sigma


In [6]:
cor = jdata['correlators']['m'] #3x3 matrix of correlators
cor.keys()

<KeysViewHDF5 ['elem_0_0', 'elem_0_1', 'elem_0_2', 'elem_1_0', 'elem_1_1', 'elem_1_2', 'elem_2_0', 'elem_2_1', 'elem_2_2']>

In [7]:
#While the matrix ostensibly contains 3x3 correlators, only those listed above are actually populated
#the reason is that the correlator matrix is symmetric so we only need to store the upper-triangular part
#For example for PiPiGnd PiPiGnd  (0,0)
cor = jdata['correlators']['m']['elem_0_0']['series']['elem_1']  #timeslice 1
t = cor.attrs['first'] #confirm time from data
data = cor['second'].attrs['data']  #note this has been blocked over 8 samples so we expect only 741//8 = 92 samples
print(t,len(data))    

#but for Sigma PiPiGnd (2,0)
jdata['correlators']['m']['elem_2_0']['series'].keys() #unpopulated



[1.] 92


<KeysViewHDF5 []>

In [8]:
#Lets get the jackknife data now
#we'll store just the upper triangular part of the correlator matrix
Lt=64
block_size = 8
nsample_unblocked = 741
nblock = nsample_unblocked // block_size
jdata = f['j_data']
jdata_cors = [ [None for j in range(3)] for i in range(3)] #3 ops
for i in range(3):
    for j in range(i,3):
        jdata_cors[i][j] = sl.CorrelationFunction(Lt)
        cdata = jdata['correlators']['m']["elem_%d_%d" % (i,j)]['series']
        for t in range(Lt):
            jvals = cdata["elem_%d" % t]['second'].attrs['data']
            assert len(jvals) == nblock
            jdist = sl.JackknifeDistribution(nblock)
            for s in range(nblock):
                jdist[s] = jvals[s]                        
            print(i,j,t,jdist)
            jdata_cors[i][j].setValue(t, jdist)
            jdata_cors[i][j].setCoord(t, float(t))


0 0 0 1720533968568.517334 +- 5152527015.142453
0 0 1 1308082730816.283691 +- 3838743441.830425
0 0 2 817893517712.709229 +- 2863212729.101227
0 0 3 546447262835.104431 +- 2279669730.728789
0 0 4 371442428617.719116 +- 1977207304.061023
0 0 5 255425904124.481415 +- 1755689182.491866
0 0 6 176186513933.533051 +- 1675816329.593119
0 0 7 121975341117.497650 +- 1675622507.859246
0 0 8 84486142123.333389 +- 1765400455.676582
0 0 9 58249791177.174446 +- 1879393121.904631
0 0 10 39940393601.323479 +- 1958523855.933918
0 0 11 27648826491.399117 +- 1990866054.886972
0 0 12 18821224075.937077 +- 2002394932.749103
0 0 13 12563605837.880201 +- 1979220601.150931
0 0 14 8421808271.079186 +- 1890603345.313394
0 0 15 6006876270.216669 +- 1801576701.302305
0 0 16 4230730202.962997 +- 1728716806.666870
0 0 17 3130040947.329866 +- 1666463901.359768
0 0 18 2141599622.018576 +- 1550578630.086805
0 0 19 1759634993.427925 +- 1447183512.452421
0 0 20 1391825459.095830 +- 1405494121.983228
0 0 21 1310339203.70

In [9]:
#For block double jackknife we expect a flattened array of size nblock * ( block_size*nblock - block_size ) = 66976
nouter_samp = nblock
ninner_samp = block_size*nblock - block_size
nunrolled= nouter_samp * ninner_samp
bdjdata = f['bdj_data']
bdjdata_cors = [ [None for j in range(3)] for i in range(3)]  #3 ops
for i in range(3):    
    for j in range(i,3):
        cdata = bdjdata['correlators']['m']["elem_%d_%d" % (i,j)]['series']   
        bdjdata_cors[i][j] = sl.CorrelationFunction(Lt)
        for t in range(Lt):
            jvals = np.array(cdata["elem_%d" % t]['second']['data']['unrolled_data'])
            assert len(jvals) == nunrolled
            jdist = sl.BlockDoubleJackknifeDistribution(nsample_unblocked, block_size)
            assert jdist.size() == nouter_samp and jdist[0].size() == ninner_samp
            u = 0
            for so in range(nouter_samp):
                jdist[so].sampleVector()[:] = jvals[u:u+ninner_samp]
                u+=ninner_samp
                #for si in range(ninner_samp):
                #    jdist[so][si] = jvals[u]
                #    u+=1
            print(i,j,t,jdist[0])
            bdjdata_cors[i][j].setValue(t, jdist)
            bdjdata_cors[i][j].setCoord(t, float(t))
            

0 0 0 1719845318663.085205 +- 3798209038.402640
0 0 1 1307820430212.833252 +- 2909128936.388778
0 0 2 817699872842.041504 +- 2226474219.047954
0 0 3 546388343762.084473 +- 1891743760.051520
0 0 4 371412451290.504578 +- 1709324819.823354
0 0 5 255402156926.810364 +- 1594969223.744068
0 0 6 176168180006.580048 +- 1544164369.009320
0 0 7 121926989500.236908 +- 1539336947.822167
0 0 8 84393148446.034088 +- 1553080940.401558
0 0 9 58106579984.547539 +- 1580894996.621566
0 0 10 39740757238.679298 +- 1596126764.802656
0 0 11 27458288311.910011 +- 1605923635.424059
0 0 12 18649288516.866985 +- 1586350433.657506
0 0 13 12436689759.691626 +- 1558205219.299251
0 0 14 8320147316.048739 +- 1516845796.658208
0 0 15 5940132342.937119 +- 1479817202.220709
0 0 16 4193917768.962172 +- 1449018489.926064
0 0 17 3090201649.459730 +- 1440240086.797071
0 0 18 2086592091.158474 +- 1426946471.970658
0 0 19 1667299566.862962 +- 1429892895.633361
0 0 20 1301633731.376409 +- 1435983835.335167
0 0 21 1202635314.52

In [10]:
#Define the fit function
#All data are cosh-like, but due to the two single-pion operators being separated by 4 timeslices the backwards-propagating component
#needs a shift
#We also factor out an overall scale to keep the output numbers more readable
class FitTwoPionCosh:
    def __init__(self,nstate,Lt,Ascale,dt,label):
        self.Lt = Lt
        self.Ascale = Ascale
        self.dt = dt
        self.nstate = nstate
        self.label = label
    def value(self,t,params):
        assert len(params) == self.nparam()
        out = None
        for s in range(self.nstate):
            poff = 3*s
            A0 = params[poff]
            A1 = params[poff+1]
            E = params[poff+2]
            v = A0 * A1 * self.Ascale * ( np.exp(-E*t) + np.exp(-E*(self.Lt-self.dt-t)) ) 
            if s==0:
                out = v
            else:
                out += v
        return out
    
    def deriv(self,t,params):
        assert len(params) == self.nparam()
        T=len(t)
        out = np.zeros((T,3*self.nstate))
        for s in range(self.nstate):
            poff = 3*s
            A0 = params[poff]
            A1 = params[poff+1]
            E = params[poff+2]
            out[:,poff] = A1 * self.Ascale * ( np.exp(-E*t) + np.exp(-E*(self.Lt-self.dt-t)) ) 
            out[:,poff+1] = A0 * self.Ascale * ( np.exp(-E*t) + np.exp(-E*(self.Lt-self.dt-t)) ) 
            out[:,poff+2] = -t* A0 * A1 * self.Ascale * np.exp(-E*t) \
                            - (self.Lt-self.dt - t)* A0 * A1 * self.Ascale*np.exp(-E*(self.Lt-self.dt-t))
        return out

    def nparam(self):
        return 3*self.nstate


In [11]:
#Define the full set of parameters
nop=3
nstate = 2
param_map = dict()

#Amplitudes
poff = 0
for op in range(nop):
    for s in range(nstate):
        param_map["A_%d_%d" % (op,s)] = poff
        poff+=1
#Energies
for s in range(nstate):
    param_map["E_%d" % s] = poff
    poff += 1

param_map

{'A_0_0': 0,
 'A_0_1': 1,
 'A_1_0': 2,
 'A_1_1': 3,
 'A_2_0': 4,
 'A_2_1': 5,
 'E_0': 6,
 'E_1': 7}

In [12]:
#Define the dt values for each operator
dtvals = [4,4,0]

In [13]:
#Build the multifit function
#We will index operator pairs (op1,op2) as op2+nop*op1
#For convenience we will setup fitfuncs for all nop*nop operator combinations even though we will only use the upper-triangular ones
Ascale = 1.0e13
fitfunc = sl.FitMulti()
for op1 in range(nop):
    for op2 in range(nop):
        #Compute the parameter map
        #Aop1_0, Aop2_0, E_0,  Aop1_1, Aop2_1, E_1 ...
        fpmap = [0 for i  in range(3*nstate)]
        for s in range(nstate):
            poff = 3*s
            fpmap[poff] = param_map["A_%d_%d" % (op1,s)]
            fpmap[poff+1] = param_map["A_%d_%d" % (op2,s)]
            fpmap[poff+2] = param_map["E_%d" % s]
        dt = dtvals[op1] + dtvals[op2] #total displacement for both operators
        print(op1,op2,fpmap," dt=", dt)
        ff = FitTwoPionCosh(nstate,Lt,Ascale,dt,"fit_%d_%d" % (op1,op2))
        fitfunc.addFitFunc(ff, fpmap)

0 0 [0, 0, 6, 1, 1, 7]  dt= 8
0 1 [0, 2, 6, 1, 3, 7]  dt= 8
0 2 [0, 4, 6, 1, 5, 7]  dt= 4
1 0 [2, 0, 6, 3, 1, 7]  dt= 8
1 1 [2, 2, 6, 3, 3, 7]  dt= 8
1 2 [2, 4, 6, 3, 5, 7]  dt= 4
2 0 [4, 0, 6, 5, 1, 7]  dt= 4
2 1 [4, 2, 6, 5, 3, 7]  dt= 4
2 2 [4, 4, 6, 5, 5, 7]  dt= 0


In [14]:
print(fitfunc.nparam())
assert fitfunc.nparam() == len(param_map)

8


In [15]:
#Build the combined correlation function
ninput_cor = 0
for op1 in range(nop):
    for op2 in range(op1,nop):
        print(idx_op_map[op1], idx_op_map[op2])
        ninput_cor += 1
jdata_all = sl.CorrelationFunction(ninput_cor*Lt)
bdjdata_all = sl.CorrelationFunction(ninput_cor*Lt)
idx=0
for op1 in range(nop):
    for op2 in range(op1,nop):
        for t in range(Lt):
            fidx= op2+3*op1
            assert fitfunc.fitfuncs[fidx].label == "fit_%d_%d" % (op1,op2)
            jdata_all.setCoord(idx, (op2+3*op1, float(t)) )
            jdata_all.setValue(idx, jdata_cors[op1][op2].value(t))
            print(jdata_all.coord(idx), "%10.5E" % jdata_all.value(idx).mean(), "%10.5E" % jdata_all.value(idx).standardError())      

            bdjdata_all.setCoord(idx, (op2+3*op1, float(t)) )
            bdjdata_all.setValue(idx, bdjdata_cors[op1][op2].value(t))
            
            idx+=1
                    


PiPiGnd PiPiGnd
PiPiGnd PiPiExc
PiPiGnd Sigma
PiPiExc PiPiExc
PiPiExc Sigma
Sigma Sigma
(0, 0.0) 1.72053E+12 5.15253E+09
(0, 1.0) 1.30808E+12 3.83874E+09
(0, 2.0) 8.17894E+11 2.86321E+09
(0, 3.0) 5.46447E+11 2.27967E+09
(0, 4.0) 3.71442E+11 1.97721E+09
(0, 5.0) 2.55426E+11 1.75569E+09
(0, 6.0) 1.76187E+11 1.67582E+09
(0, 7.0) 1.21975E+11 1.67562E+09
(0, 8.0) 8.44861E+10 1.76540E+09
(0, 9.0) 5.82498E+10 1.87939E+09
(0, 10.0) 3.99404E+10 1.95852E+09
(0, 11.0) 2.76488E+10 1.99087E+09
(0, 12.0) 1.88212E+10 2.00239E+09
(0, 13.0) 1.25636E+10 1.97922E+09
(0, 14.0) 8.42181E+09 1.89060E+09
(0, 15.0) 6.00688E+09 1.80158E+09
(0, 16.0) 4.23073E+09 1.72872E+09
(0, 17.0) 3.13004E+09 1.66646E+09
(0, 18.0) 2.14160E+09 1.55058E+09
(0, 19.0) 1.75963E+09 1.44718E+09
(0, 20.0) 1.39183E+09 1.40549E+09
(0, 21.0) 1.31034E+09 1.47023E+09
(0, 22.0) 8.85782E+08 1.56142E+09
(0, 23.0) 1.00507E+09 1.72096E+09
(0, 24.0) 9.00226E+08 1.93618E+09
(0, 25.0) 1.24053E+09 2.17412E+09
(0, 26.0) 1.63987E+09 2.41085E+09
(0, 

In [16]:
#Extract data in fit range
tmin=6
tmax=15
lb = lambda c: True if c[1] >= tmin and c[1] <= tmax else False
jdata_inrange = jdata_all.subset(lb)
bdjdata_inrange = bdjdata_all.subset(lb)

#bdjdata_all = sl.CorrelationFunction(ninput_cor*Lt)
print("Data in fit:")
for i in range(jdata_inrange.size()):
    c=jdata_inrange.coord(i)
    op2 = c[0] % 3
    op1 = c[0] // 3    
    print(idx_op_map[op1],idx_op_map[op2],c[1], \
          "%10.5E" % jdata_inrange.value(i).mean(), \
          "%10.5E" % jdata_inrange.value(i).standardError())
    


Data in fit:
PiPiGnd PiPiGnd 6.0 1.76187E+11 1.67582E+09
PiPiGnd PiPiGnd 7.0 1.21975E+11 1.67562E+09
PiPiGnd PiPiGnd 8.0 8.44861E+10 1.76540E+09
PiPiGnd PiPiGnd 9.0 5.82498E+10 1.87939E+09
PiPiGnd PiPiGnd 10.0 3.99404E+10 1.95852E+09
PiPiGnd PiPiGnd 11.0 2.76488E+10 1.99087E+09
PiPiGnd PiPiGnd 12.0 1.88212E+10 2.00239E+09
PiPiGnd PiPiGnd 13.0 1.25636E+10 1.97922E+09
PiPiGnd PiPiGnd 14.0 8.42181E+09 1.89060E+09
PiPiGnd PiPiGnd 15.0 6.00688E+09 1.80158E+09
PiPiGnd PiPiExc 6.0 -1.52834E+09 4.46312E+08
PiPiGnd PiPiExc 7.0 -8.85098E+08 4.58427E+08
PiPiGnd PiPiExc 8.0 -6.71861E+08 4.88938E+08
PiPiGnd PiPiExc 9.0 -6.20192E+08 5.18870E+08
PiPiGnd PiPiExc 10.0 -6.34045E+08 5.37660E+08
PiPiGnd PiPiExc 11.0 -5.17405E+08 5.36487E+08
PiPiGnd PiPiExc 12.0 -4.85675E+08 5.37361E+08
PiPiGnd PiPiExc 13.0 -5.44530E+08 5.32469E+08
PiPiGnd PiPiExc 14.0 -5.02395E+08 5.12771E+08
PiPiGnd PiPiExc 15.0 -2.76354E+08 4.86897E+08
PiPiGnd Sigma 6.0 -1.80818E+08 4.72353E+06
PiPiGnd Sigma 7.0 -1.32420E+08 4.74307E+06

In [17]:
fitter = sl.Fitter(fitfunc)
fitter.generateCovarianceMatrix(bdjdata_inrange)

In [18]:
#Guesses for params
params = [None for i in range(fitfunc.nparam())]
params[param_map["A_0_0"]] = sl.JackknifeDistribution(nblock,0.35) 
params[param_map["A_0_1"]] = sl.JackknifeDistribution(nblock,0.17) 
params[param_map["A_1_0"]] = sl.JackknifeDistribution(nblock,0.004) 
params[param_map["A_1_1"]] = sl.JackknifeDistribution(nblock,-0.05)
params[param_map["A_2_0"]] = sl.JackknifeDistribution(nblock,-4e-4) 
params[param_map["A_2_1"]] = sl.JackknifeDistribution(nblock,3e-4)
params[param_map["A_1_0"]] = sl.JackknifeDistribution(nblock,2.3e-3) 
params[param_map["E_0"]] = sl.JackknifeDistribution(nblock,0.35)
params[param_map["E_1"]] = sl.JackknifeDistribution(nblock,0.52)
for p in params:
    print(p)

0.350000 +- 0.000000
0.170000 +- 0.000000
0.002300 +- 0.000000
-0.050000 +- 0.000000
-0.000400 +- 0.000000
0.000300 +- 0.000000
0.350000 +- 0.000000
0.520000 +- 0.000000


In [22]:
chisq, dof = fitter.fit(params, jdata_inrange, ftol=1e-15, xtol=1e-15, factor=10, maxfev=10000 )

Performing a fit with 8 free parameters and 52 degrees of freedom


In [23]:
chisq.mean()/dof

np.float64(1.2982830408366821)

In [25]:
chisq.standardError()/dof

np.float64(0.345381697699792)

In [37]:
#These are the jackknife samples from my C++ code
expect = [0.347777, 0.348029, 0.347956, 0.347924, 0.348396, 0.347881, 0.347743, 0.347877, 0.348077, 0.347874, 0.348145, 0.347981, \
0.347802, 0.347994, 0.347795, 0.347985, 0.347858, 0.348165, 0.347989, 0.347743, 0.347875, 0.347734, 0.347992, 0.34802, 0.347853, \
0.348052, 0.347848, 0.347998, 0.347698, 0.348107, 0.347902, 0.348046, 0.347862, 0.347876, 0.3481, 0.347685, 0.347883, 0.348055, \
0.347968, 0.348065, 0.348069, 0.347874, 0.34788, 0.347782, 0.348037, 0.347996, 0.34785,  0.348032, 0.347928,  0.348138, 0.347915, \
0.347922, 0.347967, 0.347879,  0.348019,  0.347936, 0.347998, 0.347853, 0.347981, 0.347891, 0.348081, 0.347975, 0.348006, 0.347974,\
0.348047, 0.347981, 0.348095, 0.347817,  0.347986, 0.347955, 0.347939, 0.347958, 0.347909, 0.34797, 0.347973, 0.34783, 0.348125,\
0.347708, 0.347894, 0.347915, 0.347976, 0.34805, 0.347981, 0.347899, 0.348068, 0.348155, 0.347863, 0.347931, 0.347876, 0.347829,\
0.348003,0.347929]

#Compare
for i in range(nblock):
    abs_dif = (expect[i] - params[6].sampleVector()[i])
    rel_dif = abs_dif / expect[i]
    if abs(rel_dif) > 1e-4:
        print(i, params[6].sampleVector()[i], expect[i], rel_dif, abs_dif)

3 0.3478338586028416 0.347924 0.00025908358480129367 9.01413971584053e-05
5 0.34776958206168707 0.347881 0.0003202760090747408 0.0001114179383129299
6 0.34778885404326626 0.347743 -0.0001318618728953157 -4.585404326623577e-05
9 0.34792486094336383 0.347874 -0.0001462050724222489 -5.086094336381741e-05
14 0.34784204379197153 0.347795 -0.0001352629910479241 -4.704379197151276e-05
20 0.3477849608080187 0.347875 0.0002588262795006998 9.003919198130594e-05
24 0.3478018451800282 0.347853 0.0001470587287498178 5.115481997181037e-05
27 0.34790130068482966 0.347998 0.00027787319228935847 9.669931517031216e-05
29 0.348150186449132 0.348107 -0.00012406084661320288 -4.318644913198222e-05
30 0.3478611045913066 0.347902 0.00011754864500174675 4.0895408693397695e-05
31 0.3481075776322703 0.348046 -0.00017692383268381483 -6.157763227027102e-05
39 0.34812989423011514 0.348065 -0.00018644284864933612 -6.489423011513118e-05
43 0.34786466617181816 0.347782 -0.00023769537186565094 -8.266617181817981e-05
45

In [24]:
#The fit results
#From previous analysis, expect
#Apipi_gnd_0 = (0.368302 +- 0.00328939)
#Apipi_gnd_1 = (0.170902 +- 0.00994209)
#Apipi_exc_0 = (0.00378407 +- 0.000338216)
#Apipi_exc_1 = (-0.0513649 +- 0.00293367)
#Asigma_0 = (-0.000430964 +- 4.37699e-06)
#Asigma_1 = (0.00031514 +- 1.82108e-05)
#E0 = (0.347948 +- 0.00111266)
#E1 = (0.569174 +- 0.0139889)

for p in param_map.keys():
    idx = param_map[p]
    print(p,params[idx])


A_0_0 0.368141 +- 0.003513
A_0_1 0.169860 +- 0.010265
A_1_0 0.003834 +- 0.000358
A_1_1 -0.050422 +- 0.003165
A_2_0 -0.000431 +- 0.000005
A_2_1 0.000310 +- 0.000019
E_0 0.347944 +- 0.001157
E_1 0.563936 +- 0.015290
